# Bagging Exercise

In this exercise, you will explore the concept of Bagging (Bootstrap Aggregating) and implement it using a random forest model. Bagging is an ensemble technique mainly used for reducing the variance of a predictive model and preventing overfitting. The main idea behind bagging is to combine multiple learners in a way that the ensemble model performs better than an individual model.

## Dataset
We will use the Iris dataset for this exercise. The Iris dataset is a classic dataset from the field of machine learning, containing measurements for iris flowers of three different species. **Feel free to use another dataset!!**

## Task
Your task is to:
1. Load the dataset.
2. Preprocess the data (if necessary).
3. Implement Bagging models.
4. Evaluate the models performance.

Please fill in the following code blocks to complete the exercise.


In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Load the dataset


In [2]:
X,y = load_iris(return_X_y=True)

# Preprocess the data (if necessary)

In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the Dataset

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and Train the Classifiers

## Random Forest
Initialize and train a Random Forest classifier.

In [5]:
random_forest_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
random_forest_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

### Evaluate the model performance

In [6]:
predictions = random_forest_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Random Forest Model Accuracy: {accuracy * 100:.2f}%')

Random Forest Model Accuracy: 100.00%


## Bagging Meta-estimator
Initialize a K-Nearest Neighbors classifier and use it as the base estimator for the Bagging classifier.

In [7]:
base_estimator = KNeighborsClassifier()
bagging_classifier = BaggingClassifier(base_estimator, n_estimators=50, random_state=42)

bagging_classifier.fit(X_train, y_train)

BaggingClassifier(estimator=KNeighborsClassifier(), n_estimators=50,
                  random_state=42)

### Evaluate the model performance

In [8]:
predictions = bagging_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Bagging Classifier Model Accuracy: {accuracy * 100:.2f}%')

Bagging Classifier Model Accuracy: 100.00%


## Pasting
Initialize a Decision Tree classifier and use it as the base estimator for a Bagging classifier with Pasting (without replacement).

In [9]:
base_estimator = DecisionTreeClassifier()
pasting_classifier = BaggingClassifier(base_estimator, n_estimators=50, max_samples=0.7, bootstrap=False, random_state=42)

pasting_classifier.fit(X_train, y_train)

BaggingClassifier(bootstrap=False, estimator=DecisionTreeClassifier(),
                  max_samples=0.7, n_estimators=50, random_state=42)

### Evaluate the model performance

In [10]:
predictions = pasting_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Pasting Classifier Model Accuracy: {accuracy * 100:.2f}%')

Pasting Classifier Model Accuracy: 100.00%


## Roughly Balanced Bagging (RBB)
Implement Roughly Balanced Bagging by manually creating balanced bootstrap samples and aggregating predictions from multiple Decision Tree classifiers.

In [12]:
n_estimators = 100

ensemble_preds = np.zeros((n_estimators, len(X_test)))
ensemble_models = []

for i in range(n_estimators):
    two_indices = np.where(y_train == 2)[0]
    one_indices = np.where(y_train == 1)[0]
    zero_indices = np.where(y_train == 0)[0]

    chosen_2_indices = np.random.choice(two_indices, size=len(two_indices), replace=True)
    chosen_1_indices = np.random.choice(one_indices, size=len(one_indices), replace=True)
    chosen_0_indices = np.random.choice(zero_indices, size=len(zero_indices), replace=True)

    balanced_sample_indices = np.concatenate([chosen_2_indices, chosen_1_indices,chosen_0_indices])
    np.random.shuffle(balanced_sample_indices)

    X_train_balanced = X_train[balanced_sample_indices]
    y_train_balanced = y_train[balanced_sample_indices]

    tree_clf = DecisionTreeClassifier(random_state=i)
    tree_clf.fit(X_train_balanced, y_train_balanced)
    ensemble_models.append(tree_clf)

    ensemble_preds[i] = tree_clf.predict(X_test)


### Evaluate the model performance

In [13]:
# Majority voting across all estimators for the final prediction
final_preds = np.round(np.mean(ensemble_preds, axis=0))

# Calculate accuracy
accuracy = accuracy_score(y_test, final_preds)
print(f'Roughly Balanced Bagging Model Accuracy: {accuracy:.2f}')

Roughly Balanced Bagging Model Accuracy: 1.00
